In [1]:

%pprint

Pretty printing has been turned OFF



----
# Find Basic Qualifications Section in a File

In [2]:

%%time
%matplotlib inline
%run ../py/chart_utls.py
%run ../load_magic/charts.py
import random

qu = ChartUtilities()
dir()

Wall time: 11 s


['ChartUtilities', 'FACEBOOK_ASPECT_RATIO', 'HTML', 'In', 'Out', 'TWITTER_ASPECT_RATIO', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'ball_and_chain', 'colormaps_list', 'cycler', 'cypher_utlis', 'display', 'exit', 'first_order_linear_scatterplot', 'get_color_cycler', 'get_inf_nan_mask', 'get_ipython', 'html_analysis', 'np', 'os', 'pd', 'pearsonr', 'plt', 'qu', 'quit', 'r', 'random', 'save_fig_as_various', 'set_matplotlib_formats', 'sns', 'storage', 'to_hex']

In [5]:

cypher_str = '''
    MATCH (pos:PartsOfSpeech {is_header: 'True'})
    RETURN
        pos.pos_symbol as pos_symbol,
        pos.pos_explanation as pos_explanation,
        pos.is_task_scope as is_task_scope,
        pos.is_minimum_qualification as is_minimum_qualification,
        pos.is_preferred_qualification as is_preferred_qualification,
        pos.is_legal_notification as is_legal_notification,
        pos.is_job_title as is_job_title,
        pos.is_office_location as is_office_location,
        pos.is_job_duration as is_job_duration,
        pos.is_supplemental_pay as is_supplemental_pay,
        pos.is_educational_requirement as is_educational_requirement,
        pos.is_interview_procedure as is_interview_procedure,
        pos.is_corporate_scope as is_corporate_scope,
        pos.is_posting_date as is_posting_date,
        pos.is_other as is_other;'''
pos_df = pd.DataFrame(qu.cu.get_execution_results(cypher_str, verbose=False))
color_cycler = get_color_cycler(pos_df.shape[0])
header_pos_list = pos_df.pos_symbol.tolist()
rgba_dict = {}
for header_pos_symbol, face_color_dict in zip(header_pos_list, color_cycler()):
    face_color = face_color_dict['color']
    rgba_dict[header_pos_symbol] = face_color

In [6]:

for header_pos_symbol, face_color in rgba_dict.copy().items():
    nonheader_pos_symbol = 'O-' + header_pos_symbol.split('-')[1]
    fc_array = face_color.copy()
    fc_array[-1] = 0.5
    rgba_dict[nonheader_pos_symbol] = fc_array

In [7]:

from matplotlib.colors import to_hex
from IPython.display import HTML, display

html_str = ''
for pos, rgba in rgba_dict.items():
    hex_str = to_hex(rgba, keep_alpha=True)
    html_str += f'<span style="color:{hex_str};">{pos}</span>'
    if pos == 'H-O':
        html_str += '<br />'
    else:
        html_str += '&nbsp;'
display(HTML(html_str))

In [8]:

%%time
HEADER_PATTERN_DICT = qu.s.load_object('HEADER_PATTERN_DICT')
file_name = random.choice(list(HEADER_PATTERN_DICT.keys())).strip()
qu.visualize_cfr_child_str_predictions(file_name=file_name)

Wall time: 457 ms


['O', 'O', 'O', 'O', 'O', 'O', 'H', 'O', 'O', 'O', 'O-TS', 'O', 'O', 'O', 'O', 'H', 'O', 'O', 'H', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'H-RQ', 'O', 'O', 'O', 'O', 'O', 'O', 'H-SP', 'O-SP', 'O', 'O', 'O', 'H-JD', 'O-OL', 'H-RQ', 'O', 'H-OL', 'O', 'H-OL', 'O']

In [9]:

%run ../py/cypher_utlis.py
cu = CypherUtilities()

In [10]:

row_objs_list = cu.get_execution_results(cu.select_filename_isheader_cypher_str, verbose=False)
is_header_df = pd.DataFrame(row_objs_list)
def f(df):
    return df.is_header.str.match('(True|False)').all(skipna=False)
s = is_header_df.groupby('file_name').apply(f)
file_names_list = s[s==True].index.tolist()

In [11]:

import random

file_name = random.choice(file_names_list)
NAVIGABLE_PARENT_IS_HEADER_DICT = cu.s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
child_strs_list = cu.get_child_strs_from_file(file_name)

In [21]:

%run ../py/html_analysis.py
hc = HeaderCategories()
crf = CrfUtilities()
feature_dict_list = HEADER_PATTERN_DICT[file_name]
feature_tuple_list = [hc.get_feature_tuple(feature_dict) for feature_dict in feature_dict_list]
prediction_dict_list = crf.CRF.predict_marginals_single(crf.sent2features(feature_tuple_list))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

AttributeError: 'LrUtilities' object has no attribute 'lda_predict_percent'